In [7]:
#import
import math
import torch

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import lightning.pytorch as pl
import torch.nn.functional as F

from sklearn.metrics import mean_squared_error
from lightning.pytorch.loggers import CSVLogger
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

In [2]:
#make screen wide
from IPython.core.display import display, HTML
from IPython import get_ipython as get_ipython
display(HTML(
    '<style>'
        '#notebook { padding-top:0px !important;margin-left:-30px !important; } ' 
        '.container { width:100% !important; } '
        '.end_space { min-height:0px !important; } '
    '</style>'
))

# FCNN

In [9]:
logger_path = './csv_logs'
data_path = '../data/clasdb_pi_plus_n.txt'

name_of_the_experiment = 'first'
logger_version = '1.0001'


scale_data = False
test_size = 0.1
batch_size = 16

hyperparams_dict = {
    'scale_data': scale_data,
    'test_size': test_size,
    'batch_size': batch_size,
    
}

In [10]:
logger = CSVLogger(save_dir=logger_path, 
                   name=name_of_the_experiment,
                   version=logger_version)

logger.log_hyperparams()

In [3]:
class InterpolDataSet(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
        self.len = len(labels)

    def __getitem__(self, index):
        feature = self.features[index]
        label = self.labels[index]
        return feature, label

    def __len__(self):
        return self.len

class InterpolDataModule(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
        self.df = None
    
    def setup(self, stage):
        # data reading and preprocessing
        df = pd.read_csv(data_path, delimiter='\t', header=None)
        df.columns = ['Ebeam', 'W', 'Q2', 'cos_theta', 'phi', 'dsigma_dOmega', 'error', 'id']

        df.loc[8314:65671, 'Ebeam'] = 5.754 # peculiarity of this dataset.
        df['phi'] = df.phi.apply(lambda x: math.radians(x))
        df = df.drop('id', axis=1)
        df = df.iloc[df[['Ebeam', 'W', 'Q2', 'cos_theta', 'phi']].drop_duplicates().index]
        self.df = df
        
        #train test split
        feature_data = df[['Ebeam', 'W', 'Q2', 'cos_theta', 'phi']]
        label_data = df['dsigma_dOmega']
        
        if scale_data:
            scaler_feature = StandardScaler()
            scaler_target = StandardScaler()
            feature_data = scaler_feature.fit_transform(feature_data)
            label_data = scaler_target.fit_transform(label_data.values.reshape(-1,1))
        else:
            pass
        
        train_feature_data, test_feature_data, train_label_data, test_label_data = train_test_split(feature_data, label_data, 
                                                                                                    test_size=test_size, random_state=1438)
        
        
        self.train_dataset = InterpolDataSet(torch.tensor(train_feature_data.values, dtype=torch.float32), 
                                             torch.tensor(train_label_data.values, dtype=torch.float32))
        self.test_dataset = InterpolDataSet(torch.tensor(test_feature_data.values, dtype=torch.float32), 
                                            torch.tensor(test_label_data.values, dtype=torch.float32))

    def train_dataloader(self):
        return DataLoader(dataset = self.train_dataset, batch_size = batch_size, shuffle = False, num_workers=0)

    def val_dataloader(self):
        return DataLoader(dataset = self.test_dataset, batch_size = batch_size, shuffle = False, num_workers=0)

In [5]:
class InterpolRegressor(pl.LightningModule):
    def __init__(self):
        super(InterpolRegressor, self).__init__()
        
        input_dim = 5
        hidden_dim_1 = 60
        hidden_dim_2 = 80
        hidden_dim_3 = 100
        hidden_dim_4 = 120
        hidden_dim_5 = 140
        hidden_dim_6 = 140
        hidden_dim_7 = 140
        hidden_dim_8 = 140
        hidden_dim_9 = 100
        hidden_dim_10 = 80
        hidden_dim_11 = 60
        output_dim = 1
        
        self.net = nn.Sequential(
                    nn.Linear(input_dim, hidden_dim_1),
                    nn.ReLU(),
                    nn.Linear(hidden_dim_1, hidden_dim_2),
                    nn.ReLU(),
                    nn.Linear(hidden_dim_2, hidden_dim_3),
                    nn.ReLU(),
                    nn.Linear(hidden_dim_3, hidden_dim_4),
                    nn.ReLU(),
                    nn.Linear(hidden_dim_4, hidden_dim_5),
                    nn.ReLU(),
                    nn.Linear(hidden_dim_5, hidden_dim_6),
                    nn.ReLU(),
                    nn.Linear(hidden_dim_6, hidden_dim_7),
                    nn.ReLU(),
                    nn.Linear(hidden_dim_7, hidden_dim_8),
                    nn.ReLU(),
                    nn.Linear(hidden_dim_8, hidden_dim_9),
                    nn.ReLU(),
                    nn.Linear(hidden_dim_9, hidden_dim_10),
                    nn.ReLU(),
                    nn.Linear(hidden_dim_10, hidden_dim_11),
                    nn.ReLU(),
                    nn.Linear(hidden_dim_11, output_dim)
                )

    def forward(self, x):
        return self.net(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = nn.MSELoss()
        loss = loss(y_hat.reshape(-1), y)
        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        global ta
        ta = batch
        x, y = batch
        y_hat = self.forward(x)
        loss = nn.MSELoss()
        loss = loss(y_hat.reshape(-1), y)
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer

In [6]:
data_module = InterpolDataModule()
model = InterpolRegressor()

early_stop_callback = [EarlyStopping(monitor="val_loss", mode="min", min_delta=0.005, patience=2, verbose=True)]
trainer = pl.Trainer(max_epochs=1000,
                     accelerator='cpu',
                     callbacks=early_stop_callback)
trainer.fit(model, data_module)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 128 K 
------------------------------------
128 K     Trainable params
0         Non-trainable params
128 K     Total params
0.515     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/andrey.golda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/Users/andrey.golda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

/Users/andrey.golda/.pyenv/versions/3.9.0/lib/python3.9/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [11]:
preds = model.forward(data_module.train_dataset.features)
mean_squared_error(data_module.train_dataset.labels, preds.detach().numpy())

0.8475419

In [12]:
preds = model.forward(data_module.test_dataset.features)
mean_squared_error(data_module.test_dataset.labels, preds.detach().numpy())

0.7896977

In [15]:
# Loss function
# lr
# optimizer
# architecture
# batch_size
# early_stopping - min_delta, patience